In [ ]:
import os
import sys
import serial
import threading
import datetime
import gspread
from google.oauth2.service_account import Credentials
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import queue
import csv

# Define column headers based on CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Google Sheets scopes and credentials
SCOPE = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

# Main GUI Class for FED3 Logging System
class FED3MonitorApp:

    def __init__(self, root):
        self.root = root
        self.root.title("FED3 Data Monitor")

        # GUI variables
        self.experimenter_name = tk.StringVar()
        self.experiment_name = tk.StringVar()
        self.json_path = tk.StringVar()
        self.spreadsheet_id = tk.StringVar()
        self.save_path = ""
        self.data_to_save = {}

        # Google Sheets client setup
        self.gspread_client = None

        # Queue for thread-safe serial data handling
        self.data_queue = queue.Queue()

        # GUI setup
        self.setup_gui()

    def setup_gui(self):
        # Experimenter name
        tk.Label(self.root, text="Experimenter Name:", font=("Helvetica", 10, "bold")).grid(column=0, row=0, sticky=tk.E, padx=2, pady=2)
        self.experimenter_entry = ttk.Entry(self.root, textvariable=self.experimenter_name)
        self.experimenter_entry.grid(column=1, row=0, sticky=tk.W, padx=2, pady=2)

        # Experiment name
        tk.Label(self.root, text="Experiment Name:", font=("Helvetica", 10, "bold")).grid(column=2, row=0, sticky=tk.E, padx=2, pady=2)
        self.experiment_entry = ttk.Entry(self.root, textvariable=self.experiment_name)
        self.experiment_entry.grid(column=3, row=0, sticky=tk.W, padx=2, pady=2)

        # JSON file path
        tk.Label(self.root, text="Google API JSON File:", font=("Helvetica", 10, "bold")).grid(column=0, row=1, sticky=tk.E, padx=2, pady=2)
        self.json_entry = ttk.Entry(self.root, textvariable=self.json_path)
        self.json_entry.grid(column=1, row=1, sticky=tk.W, padx=2, pady=2)
        self.browse_json_button = tk.Button(self.root, text="Browse", command=self.browse_json)
        self.browse_json_button.grid(column=2, row=1, padx=5, pady=10)

        # Google Spreadsheet ID
        tk.Label(self.root, text="Google Spreadsheet ID:", font=("Helvetica", 10, "bold")).grid(column=0, row=2, sticky=tk.E, padx=2, pady=2)
        self.spreadsheet_entry = ttk.Entry(self.root, textvariable=self.spreadsheet_id)
        self.spreadsheet_entry.grid(column=1, row=2, sticky=tk.W, padx=2, pady=2)

        # Data Folder Path
        self.browse_button = tk.Button(self.root, text="Browse Data Folder", font=("Helvetica", 15, "bold"), command=self.browse_folder, bg="gold", fg="blue")
        self.browse_button.grid(column=0, row=3, padx=5, pady=10)

        # Start button
        self.start_button = tk.Button(self.root, text="START", font=("Helvetica", 15, "bold"), bg="green", fg="white", command=self.start_logging)
        self.start_button.grid(column=1, row=4, padx=5, pady=10)

        # Stop button
        self.stop_button = tk.Button(self.root, text="STOP(SAVE & QUIT)", font=("Helvetica", 15, "bold"), bg="red", fg="white", command=self.stop_logging)
        self.stop_button.grid(column=2, row=4, padx=5, pady=10)

    def browse_json(self):
        # Open a file dialog to select JSON file
        self.json_path.set(filedialog.askopenfilename(title="Select JSON File"))

    def browse_folder(self):
        # Open a file dialog to select the folder to save data locally
        self.save_path = filedialog.askdirectory(title="Select Folder to Save Data")

    def start_logging(self):
        # Initialize Google Sheets API
        try:
            creds = Credentials.from_service_account_file(self.json_path.get(), scopes=SCOPE)
            self.gspread_client = gspread.authorize(creds)
            print(f"Connected to Google Spreadsheet: {self.spreadsheet_id.get()}")
        except Exception as e:
            messagebox.showerror("Error", f"Error setting up Google Sheets API: {e}")
            return

        # Start reading from FED3 devices and log data to Google Sheets
        print("RTFED Mode: Logging data to Google Sheets")
        self.start_data_collection()

    def start_data_collection(self):
        # Automatically detect and handle the connected FED3 devices using USB ports
        self.device_mappings = self.get_device_mappings_by_usb_port()

        # Start a separate thread for each FED3 device
        for mapping in self.device_mappings:
            serial_port = mapping['serial_port']
            port_identifier = mapping['port_identifier']
            threading.Thread(target=self.read_from_port, args=(serial_port, f"Port_{port_identifier}"), daemon=True).start()

    def get_device_mappings_by_usb_port(self):
        # Automatically map devices to ports
        device_mappings = []
        usb_port_mapping = {
            'usb-0:1.1': 'Port 1',
            'usb-0:1.2': 'Port 2',
            'usb-0:1.3': 'Port 3',
            'usb-0:1.4': 'Port 4',
        }
        for symlink in os.listdir('/dev/serial/by-path/'):
            symlink_path = os.path.join('/dev/serial/by-path/', symlink)
            serial_port = os.path.realpath(symlink_path)
            if 'ttyACM' in serial_port or 'ttyUSB' in serial_port:
                usb_port_path = self.get_usb_port_path_from_symlink(symlink)
                port_identifier = usb_port_mapping.get(usb_port_path)
                if port_identifier:
                    device_mappings.append({
                        'serial_port': serial_port,
                        'port_identifier': port_identifier,
                    })
        return device_mappings

    def get_usb_port_path_from_symlink(self, symlink):
        match = re.search(r'usb-\d+:\d+(\.\d+)*', symlink)
        return match.group() if match else None

    def read_from_port(self, port, worksheet_name):
        # Open serial port and Google Spreadsheet
        try:
            ser = serial.Serial(port, 115200, timeout=1)
            spreadsheet = self.gspread_client.open_by_key(self.spreadsheet_id.get())
            sheet = self.get_or_create_worksheet(spreadsheet, worksheet_name)

            while True:
                data = ser.readline().decode('utf-8').strip()
                data_list = data.split(",")
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
                data_list = data_list[1:]

                if len(data_list) == len(column_headers) - 1:
                    sheet.append_row([timestamp] + data_list)
                    self.data_queue.put([timestamp] + data_list)
                else:
                    print(f"Warning: Data length {len(data_list)} does not match header length")

        except Exception as e:
            print(f"Error reading from {port}: {e}")

    def get_or_create_worksheet(self, spreadsheet, title):
        # Create or retrieve a worksheet in Google Spreadsheet
        try:
            sheet = spreadsheet.worksheet(title)
            print(f"Worksheet '{title}' found.")
        except gspread.exceptions.WorksheetNotFound:
            print(f"Worksheet '{title}' not found. Creating a new one.")
            sheet = spreadsheet.add_worksheet(title=title, rows="1000", cols="20")
            sheet.append_row(column_headers)
        return sheet

    def stop_logging(self):
        # Stop all threads and save data locally
        stop_event.set()
        self.save_all_data()

    def save_all_data(self):
        # Save accumulated data locally
        current_time = datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
        for port_identifier in self.device_mappings:
            filename = f"{self.save_path}/{current_time}_{port_identifier}.csv"
            with open(filename, mode='w', newline='') as file:
                writer = csv.writer(file)
                writer.writerow(column_headers)
                while not self.data_queue.empty():
                    writer.writerow(self.data_queue.get())

# Main Execution
if __name__ == "__main__":
    root = tk.Tk()
    app = FED3MonitorApp(root)
    root.mainloop()
